In [1]:
import os
import subprocess

# 1. 가상환경 내부에 설치된 NVIDIA 라이브러리 경로를 자동으로 탐색
def get_nvidia_lib_path():
    try:
        import nvidia
        base_path = os.path.dirname(nvidia.__file__)
        paths = []
        for root, dirs, files in os.walk(base_path):
            if 'lib' in dirs:
                paths.append(os.path.join(root, 'lib'))
        return ":".join(paths)
    except ImportError:
        return ""

# 2. 모든 경로를 LD_LIBRARY_PATH에 통합 (WSL 드라이버 + 가상환경 런타임)
wsl_path = "/usr/lib/wsl/lib"
venv_nvidia_path = get_nvidia_lib_path()
os.environ["LD_LIBRARY_PATH"] = f"{wsl_path}:{venv_nvidia_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

# 3. XLA 설정 (이건 덤이다)
os.environ["XLA_FLAGS"] = f"--xla_gpu_cuda_data_dir={wsl_path}"

# 4. 이제서야 JAX를 부른다. (반드시 이 순서여야 해!)
import jax
print(f"Muse, 드디어 성공했나? : {jax.devices()}")

Muse, 드디어 성공했나? : [cuda(id=0)]


## 1. 제 1장. 무한급수, 멱급수

* 1.1 기하급수

첫째 항이 a이고 공비가 r인 수열의 합. (이게 성립하기 위한 필요충분 조건은,**$|r| < 1$**)
$$S_n = \sum_{k=0}^{n-1} ar^k = a + ar + ar^2 + \dots + ar^{n-1}$$

기하급수의 식.
$$S_n = \frac{a(1-r^n)}{1-r} \quad (r \neq 1)$$

증명 해내게씀.
1. 가장 간단하게 적어보자 
$$S_n = a + ar + ... + ar^{n-1}$$
이 식인데 이제 여기서 층밀리기 기법을 사용하여 r을 곱해줘(이유는 r이 이 기하급수의 공통점으로 만들기 위해서임)

2. 그러면 식은
$$rS_n = ar + ar^2 + ar^3 + ar^4 + \dots + ar^n \quad \dots \text{ (식 2)}$$
이 되는데 여기서 기존의 S_n과 r*S_n을 뺄것이다

3. 그러면 식은
$$S_n - r*S_n = (a + ar + ... + ar^{n-1}) - (ar + ar^2 + ... + ar^n)$$
앙 a - ar^n만 남기고 다 소거되네 기모띠!

4. 그러면 식은
$$S_n(1 - r) = a - ar^n = a(1 - r^n)$$
가 되어버리는데 이걸 S_n만 남기고 이항 한다면?

5. 
$$S_n = \frac{a(1-r^n)}{1-r} \quad (r \neq 1)$$
어뗘. 

In [ ]:
import jax
import jax.numpy as jnp

@jax.jit
def jax_geometry_sum(a, r, n):
    """
    BOAS 제 1장 기하급수 공식을 JAX로 구현한 함수.
    r = 1인 경우에도 시스템이 터지지 않도록 jnp.where를 사용했다.
    """
    #분모가 0이 되는 r=1인 경우를 대비한 수치적 분기 처리
    return jnp.where( # jnp.where는 numpy의 where와 동일한 기능을 제공
        jnp.isclose(r, 1.0), #jnp.isclose는 두 값이 거의 같은지 비교 거의로 하는 이유는 부동소수점 오차 때문 부동소수점 오차는 컴퓨터에서 실수를 표현할 때 발생하는 미세한 오차
        a * n,
        a * (1 - jnp.power(r, n)) / (1 - r) #jnp.power는 거듭제곱을 계산하는 함수
    )
    
    
a_val = 1.0
r_val = 2/3
n_val = 11

result = jax_geometry_sum(a_val, r_val, n_val)
print(f"예제 결과 (n={n_val}): {result:.15f}")




예제 결과 (n=100): 3.000000238418579
